Logs 
- [2025/08/07]    
  Get 31 bonds from the list of bonds in `./data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp`

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from docplex.mp.model_reader import ModelReader

In [2]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
  "font.size": 16,
  "grid.alpha": 0.25})

In [3]:
lp_file_path = "./data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp"

model = ModelReader.read(lp_file_path)

In [4]:
binary_var_names = []
for var in model.iter_variables():
  if var.is_binary() and var.name is not None:
    var_name = var.name.split("_")[1]

    binary_var_names.append(var_name)

binary_var_names, len(binary_var_names)

(['020002BJ9',
  '026874DS3',
  '081437AT2',
  '097023CJ2',
  '13645RAD6',
  '13645RBF0',
  '14448CBC7',
  '15135BAW1',
  '21871XAS8',
  '24422EWZ8',
  '24422EXP9',
  '314353AA1',
  '36166NAK9',
  '438516CM6',
  '443201AC2',
  '444859BR2',
  '444859BV3',
  '444859BY7',
  '444859CA8',
  '45687VAB2',
  '539830CD9',
  '540424AT5',
  '56501RAN6',
  '655844CR7',
  '655844CT3',
  '75513EAD3',
  '759351AP4',
  '760759BA7',
  '760759BC3',
  '907818FX1',
  '91324PEJ7'],
 31)

Read the spreadsheet `data_assets_dictionary`

In [5]:
df = pd.read_excel("./data/1/data_assets_dictionary.xlsx", parse_dates=["maturity"])
df = df.drop(["Unnamed: 0"], axis=1)
df

,isin,ccy,assetId,strategyName,secGroup,secType,cpn,posSource,baseCcy,krd10y,...,Large Issuer_new,issuerTicker_new,sodraw.filterLevel1_new,sodraw.filterLevel3_new,sodraw.filterLevel4_new,security.elements.creditQualityBuckets_new,security.elements.normalizedRating_new,sodraw.filterLevel2_new,assetId_new,main_key
0,US001055BJ00,USD,001055BJ0,Unassigned,BND,CORP,3.6,EOD,USD,0.000000,...,N,AFL,Investment Grade Credit,Insurance,Life Insurance,A,A,Financial,001055BJ0,VCIT | VCIT (bench)
1,US001084AS13,USD,001084AS1,Unassigned,BND,CORP,5.8,EOD,USD,4.436857,...,N,AGCO,Investment Grade Credit,Capital Goods,Construction Machinery,BBB,BBB,Industrial,001084AS1,VCIT | VCIT (bench)
2,US00108WAM29,USD,00108WAM2,Unassigned,BND,CORP,2.1,EOD,USD,0.000000,...,N,AEP,Investment Grade Credit,Electric,Other,BBB,BBB,Utilities,00108WAM2,VCIT | VCIT (bench)
3,US00108WAP59,USD,00108WAP5,Unassigned,BND,CORP,4.7,EOD,USD,0.828971,...,N,AEP,Investment Grade Credit,Electric,Other,BBB,BBB,Utilities,00108WAP5,VCIT | VCIT (bench)
4,US00108WAR16,USD,00108WAR1,Unassigned,BND,CORP,5.4,EOD,USD,2.746458,...,N,AEP,Investment Grade Credit,Electric,Other,BBB,BBB,Utilities,00108WAR1,VCIT | VCIT (bench)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,NaN,USD,USD_CCASH,Unassigned,CASH,CASH,0.0,ORD,USD,0.000000,...,N,USD,NaN,Other,Other,NR,Junk / NR,NaN,USD_CCASH,VCIT | VCIT (bench)
2625,NaN,USD,USD_CCASH,Unassigned,CASH,CASH,0.0,TRD,USD,0.000000,...,N,USD,NaN,Other,Other,NR,Junk / NR,NaN,USD_CCASH,VCIT | VCIT (bench)
2626,NaN,USD,UXYZ42029,Unassigned,FUTURE,FIN,0.0,EOD,USD,5.619551,...,N,UXYZ4,NaN,Other,Other,AAA/AA,AAA,NaN,UXYZ42029,VCIT | VCIT (bench)
2627,NaN,USD,UXYZ4202F,Unassigned,CASH,FUTURE,0.0,EOD,USD,0.000000,...,N,UXYZ4,NaN,Other,Other,AAA/AA,AAA,NaN,UXYZ4202F,VCIT | VCIT (bench)


In [10]:
df_description = pd.read_excel("./data/1/data_assets_dictionary.xlsx", sheet_name="Dictionary")
df_description

,Field,Description
0,isin,12‑character International Securities Identifi...
1,ccy,ISO‑4217 currency in which the security is den...
2,assetId,Internal master identifier used by Vanguard sy...
3,strategyName,Friendly name of the trading or optimization s...
4,secGroup,"High‑level security grouping (Government, Corp..."
...,...,...
110,Large Issuer_new / issuerTicker_new,Post‑trade or scenario “what‑if” versions of l...
111,sodraw.filterLevel1_new … sodraw.filterLevel4_new,Scenario overrides of SoDraw filter levels 1–4.
112,security.elements.creditQualityBuckets_new / s...,Scenario overrides of credit‑quality bucket an...
113,assetId_new,Temporary scenario asset ID generated by the o...


Collect all 31 rows from `df` based on `binary_var_names`

In [6]:
pd.set_option("display.max_columns", 500)

In [12]:
df_31bonds = df[df["assetId"].isin(binary_var_names)]
df_31bonds.reset_index(inplace=True)

# df_31bonds_clean = df_31bonds.drop(["ccy", "strategyName", "secGroup", "secType", "posSource", 
#   "baseCcy", "krd15y", "krd20y", "krd25y", "krd30y", "krd40y", "krd50y",
#   "portfolioName",
#   "fund_orderStatus",
#   "enriched.cashladder_tCashBalance",
#   "enriched.cashladder_tPlus1CashBalance",
#   "enriched.cashladder_tPlus2CashBalance",
#   "enriched.cashladder_tPlus3CashBalance",
#   "enriched.cashladder_tPlus4CashBalance",
#   "enriched.cashladder_tPlus5CashBalance",
#   "enriched.cashladder_tPlus6CashBalance",
#   "enriched.cashladder_tPlus7CashBalance",
#   "enriched.cashladder_tPlus30CashBalance",
#   "enriched.cashladder_tPlus60CashBalance",
#   "enriched.cashladder_tPlus90CashBalance",
#   "bvb.krd10yCtr",
#   "bvb.krd15yCtr",
#   "bvb.krd20yCtr",
#   "bvb.krd25yCtr",
#   "bvb.krd30yCtr",
#   "bvb.krd40yCtr",
#   "bvb.krd50yCtr",
#   "trades.ctrkrd10y",
#   "trades.ctrkrd15y",
#   "trades.ctrkrd20y",
#   "trades.ctrkrd25y",
#   "trades.ctrkrd30y",
#   "trades.ctrkrd40y",
#   "trades.ctrkrd50y",
#   "fund_enriched.krd15yCtr",
#   "fund_enriched.krd20yCtr",
#   "fund_enriched.krd25yCtr",
#   "fund_enriched.krd30yCtr",
#   "fund_enriched.krd40yCtr",
#   "fund_enriched.krd50yCtr",
#   "enriched.krd15yCtr_active",
#   "enriched.krd20yCtr_active",
#   "enriched.krd25yCtr_active",
#   "enriched.krd30yCtr_active",
#   "enriched.krd40yCtr_active",
#   "enriched.krd50yCtr_active",
#   "fund_enriched.fund.krd15yCtr",
#   "fund_enriched.fund.krd20yCtr",
#   "fund_enriched.fund.krd25yCtr",
#   "fund_enriched.fund.krd30yCtr",
#   "fund_enriched.fund.krd40yCtr",
#   "fund_enriched.fund.krd50yCtr",
#   "sodraw.filterLevel1",
#   "sodraw.filterLevel1_new",
#   "sodraw.structure",
#   "sodraw.pathVngcoreCCYFilterLevel1",
#   "sodraw.pathVngcoreCCYFilterLevel2",
#   "sodraw.pathVngcoreCCYFilterLevel3",
#   "sodraw.pathVngcoreCCYFilterLevel4",
#   "sodraw.pathHUBGLOBNOSTRAT1",
#   "enriched.corpActions.usageType",
#   "enriched.tsyRV.bb_spread_to_spline_cubic",
#   "enriched.tsyRV.bb_3m_zscore_sp_spline_cubic",
#   "enriched.tsyRV.spline_range",
#   "enriched.tsyRV.rtn",
#   "enriched.tsyRV.expected_move",
#   "enriched.tsyRV.asset_swap_spd_mid",
#   "enriched.tsyRV.asw_range",
#   "enriched.otr_roll.SpecialStatusCTD",
#   "enriched.otr_roll.SpecialStatusOTR",
#   "enriched.otr_roll.OTR",
#   "bench_enriched.krd15yCtr",
#   "bench_enriched.krd20yCtr",
#   "bench_enriched.krd25yCtr",
#   "bench_enriched.krd30yCtr",
#   "bench_enriched.krd40yCtr",
#   "bench_enriched.krd50yCtr",
#   "main_key"], axis=1)

# df_31bonds_clean.to_excel("./clean_data/31bonds_assets.xlsx")

source_sheets = {
  "Data": df_31bonds,
  "Dictionary": df_description
}

with pd.ExcelWriter("./clean_data/31bonds_assets.xlsx", engine="openpyxl") as writer:
  for sheet_name, df in source_sheets.items():
    df.to_excel(writer, sheet_name=sheet_name, index=False)

